<a href="https://colab.research.google.com/github/flaviohnm/projeto_bancodedados/blob/main/ocorrencia_tipo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Instalação do módulos

In [33]:
pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Bloco de Extração

In [34]:
import pandas as pd
import pandera as pa

In [35]:
df = pd.read_csv("/content/drive/MyDrive/CENIPA/ocorrencia_tipo.csv",sep=";")

## O comando "head" exibe a quantidade de linhas contida no parentesis.

In [36]:
df.head(10)

,codigo_ocorrencia1,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao
0,45331,COM PESSOAL EM VOO,OUTROS | COM PESSOAL EM VOO,OTHR
1,45332,PERDA DE CONTROLE NO SOLO,PERDA DE CONTROLE NO SOLO,LOC-G
2,45333,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP
3,45334,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
4,45390,OPERAÇÃO A BAIXA ALTITUDE,OPERAÇÃO A BAIXA ALTITUDE,LALT
5,45391,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP
6,45392,PERDA DE CONTROLE EM VOO,PERDA DE CONTROLE EM VOO,LOC-I
7,45393,EXCURSÃO DE PISTA,EXCURSÃO DE PISTA,RE
8,45393,PERDA DE CONTROLE NO SOLO,PERDA DE CONTROLE NO SOLO,LOC-G
9,45396,COM TREM DE POUSO,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP


Exibir os tipos de dados dos objetos

In [37]:
df.dtypes

codigo_ocorrencia1            int64
ocorrencia_tipo              object
ocorrencia_tipo_categoria    object
taxonomia_tipo_icao          object
dtype: object

Até esse trecho foi realizado o carregamento de dados com a conversão das colunas que contém datas para o tipo de dado "date" com o método "parse".

In [38]:
df.replace(['***','**','####','###!','****','*****','NULL'], pd.NA, inplace=True)

In [39]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia1":pa.Column(pa.Int),
        "ocorrencia_tipo":pa.Column((pa.String),nullable=True),
        "ocorrencia_tipo_categoria":pa.Column((pa.String),nullable=True),
        "taxonomia_tipo_icao":pa.Column((pa.String),nullable=True)     
       
    }
)

In [40]:
df.isnull().sum()

codigo_ocorrencia1           0
ocorrencia_tipo              1
ocorrencia_tipo_categoria    1
taxonomia_tipo_icao          1
dtype: int64

##Validação do Schema

In [41]:
schema.validate(df)

,codigo_ocorrencia1,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao
0,45331,COM PESSOAL EM VOO,OUTROS | COM PESSOAL EM VOO,OTHR
1,45332,PERDA DE CONTROLE NO SOLO,PERDA DE CONTROLE NO SOLO,LOC-G
2,45333,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP
3,45334,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
4,45390,OPERAÇÃO A BAIXA ALTITUDE,OPERAÇÃO A BAIXA ALTITUDE,LALT
...,...,...,...,...
5342,80461,COM TREM DE POUSO,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
5343,80463,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
5344,80465,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP
5345,80467,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP


##Checando se o valor da coluna "codigo ocorrência" são únicos.

In [42]:
df.ocorrencia_tipo.is_unique

False

Definindo "codigo da ocorrência" como indice

In [43]:
df.set_index('ocorrencia_tipo', inplace=True)

Consultar quantos NA tem no dataframe

In [15]:
df.isna().sum()

codigo_ocorrencia1           0
ocorrencia_tipo_categoria    1
taxonomia_tipo_icao          1
dtype: int64

In [44]:
df.isnull().sum()

codigo_ocorrencia1           0
ocorrencia_tipo_categoria    1
taxonomia_tipo_icao          1
dtype: int64